In [ ]:
import sys
from pathlib import Path

sys.path.append("..")
from ribasim_lumping import NetworkAnalysis

import pandas as pd
import numpy as np
import geopandas as gpd
from pydantic import BaseModel
import xarray as xr
import dfm_tools as dfmt
import xugrid as xu
import matplotlib.pyplot as plt
import random

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)

import networkx as nx

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
base_dir = Path("c:\\Users\\NLHARN\\Documents\\SWECO_LOKAAL\\NHI-TKI-oppervlaktewatermodule-Ribasim\\TKI-NHI-oppervlaktewatermodule\\")
simulations_dir = Path(base_dir, "d-hydro\\dhydro\\")
date_range = pd.date_range("2000-01-02 23:00", periods=9, freq="2D")

In [ ]:
# create networkanalysis
network = NetworkAnalysis()

# add data from simulation sets
network.add_data_from_simulations_set(
    set_name="winter",
    simulations_dir=Path(simulations_dir, ""),
    simulations_names=["tki_zuthpen_berkel_basis.dsproj"],
    simulations_ts=date_range,
)
network.add_data_from_simulations_set(
    set_name="zomer",
    simulations_dir=Path(simulations_dir, ""),
    simulations_names=["tki_zuthpen_berkel_basis.dsproj"],
    simulations_ts=date_range,
)

# get all network data from the schematisations
network.get_network_data()

In [ ]:
# nodes = network.dhydro_nodes.sort_values(by='mesh1d_node_x', ascending=False).iloc[:150]
nodes = network.dhydro_nodes
nodes_ids = list(nodes.mesh1d_nNodes.values)

edges = network.dhydro_edges
edges = edges[(edges["start_node_no"].isin(nodes_ids)) & 
              (edges["end_node_no"].isin(nodes_ids))]

In [ ]:
# Define node_ids on which to split
# split_node_ids = [300, 1200, 1300, 1400, 1430, 1818, 1950, 2380]
# split_node_ids = [2380]
# split_node_ids = list(network.bifurcation_points.mesh1d_nNodes.values)
split_node_ids = list(network.bifurcation_points.mesh1d_nNodes.values) + list(network.confluence_points.mesh1d_nNodes.values)
# split_node_ids = list(network.weirs.mesh1d_nNodes.values) # all weirs
# split_node_ids = random.sample(split_node_ids, 25)

# load areas units (discharge units: afwaterende eenheden)
areas_file_path = Path(base_dir, "wrij_afwateringseenheden_clip_Zutphen\\wrij_afwateringseenheden_clip_Zutphen.shp")
areas = gpd.read_file(areas_file_path)[["geometry"]].to_crs(28992)

# create basins (gdf) based on nodes, edges, split_node_ids and areas
basins, areas = network.create_basins_based_on_split_node_ids(
    nodes=nodes, 
    edges=edges, 
    split_node_ids=split_node_ids, 
    areas=areas
)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 15))
basins.plot(ax=ax, edgecolor="black", column="basin", cmap="tab20", categorical=True, alpha=0.5, linewidth=2)
areas.plot(ax=ax, edgecolor="black", facecolor='none', alpha=0.5, linewidth=0.25)
edges.plot(ax=ax, color="darkblue", linewidth=3)
network.weirs.plot(ax=ax, color="orange", edgecolor="black", linewidth=2, markersize=50, zorder=100)
nodes.loc[split_node_ids].plot(ax=ax, color="red", edgecolor="black", linewidth=2, markersize=75, zorder=100)

ax.set_title('TKI pilot-gebied Zutphen: Toepassing Lumpen', fontsize=18)
plt.axis('off')

output_dir = Path("output//test_split_network//")
if not output_dir.exists():
    output_dir.mkdir(parents=True, exist_ok=True)
figure_path = Path(output_dir, "TKI_ribasim_Zutphen_voorbeeld_lumpen.png")
plt.savefig(str(figure_path), bbox_inches='tight', dpi=300)